In [577]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import ast

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, FeatureHasher
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

In [578]:
spindtTracks = pd.read_csv("JSON Files/Exported CSV Files/spindtTracks.csv")
artists = pd.read_csv("JSON Files/Exported CSV Files/artists.csv")
songs = pd.read_csv("JSON Files/Exported CSV Files/songs.csv")

In [579]:
spindtTracks = spindtTracks.drop(['Unnamed: 0', 'available_markets', 'track_number'], axis=1)

In [580]:
artists = artists.drop(['Unnamed: 0', 'href'], axis=1)

In [581]:
songs = songs.drop('Unnamed: 0', axis=1)

In [582]:
spindtTracks

,album,artists,duration_ms,explicit,id,name,popularity
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,257920,False,51ggEZWbFP5dMdWjGCDYTk,Dunes,0
1,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,201612,False,6p5LBZkLRIeVhbLh0w7EuP,Future People,0
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,232546,True,2lrRWbiHPdBicufAqrGN3r,Don't Wanna Fight,0
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,243466,False,7Gz1adaLeIHNOqJPvfnncD,Gimme All Your Love,0
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,308066,False,17ZnveSDBpG9QtL7zLJNPy,Only For You,3
...,...,...,...,...,...,...,...
95,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,330419,False,0DzNSw9Pp1yKJZynNBM8uy,Queen of the Surface Streets,0
96,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,211226,False,6qZjm61s6u8Ead9sWxCDro,Elephant,75
97,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,252139,False,5mqr5kF7oNDdcG01qazSqK,Had Ten Dollaz,46
98,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,290106,False,1rO4Ew8dCbYWqJ3L6DmJDo,All the Sand in all the Sea,35


In [583]:
spindtSongsAlbumsArr = []
for i in spindtTracks['album']:
    spindtSongsAlbumsArr.append(ast.literal_eval(i)['name'])

In [584]:
spindtArtistsArr = []
spindtMainArtistsArr = []
for i in spindtTracks['artists']:
    spindt_artists = []
    for artist in ast.literal_eval(i):
        spindt_artists.append(artist['name'])
    spindtArtistsArr.append(artist)
    spindtMainArtistsArr.append(ast.literal_eval(i)[0]['name'])

In [585]:
spindtTracksAnalysis = spindtTracks

In [586]:
spindtTracksAnalysis['main_artist'] = spindtMainArtistsArr
spindtTracksAnalysis['album'] = spindtSongsAlbumsArr
spindtTracksAnalysis = spindtTracksAnalysis.drop('id', axis=1)

In [587]:
spindt_artistsIDs = []
for i in spindtArtistsArr:
    spindt_artistsIDs.append(i['id'])

In [588]:
spindtTracksAnalysis['id'] = spindt_artistsIDs

In [589]:
artists['followers']

followersArr = []
for i in artists['followers']:
    followersArr.append(ast.literal_eval(i)['total'])

In [590]:
artists['followers'] = followersArr

In [591]:
spindtTracksAnalysis = pd.merge(spindtTracksAnalysis, artists[['id', 'genres', 'followers']], on='id', how='left')
spindtTracksAnalysis = spindtTracksAnalysis.drop(['id', 'artists'], axis=1)

In [592]:
spindtTracksAnalysis

,album,duration_ms,explicit,name,popularity,main_artist,genres,followers
0,Sound & Color,257920,False,Dunes,0,Alabama Shakes,"['alabama indie', 'modern blues rock', 'rock']",1604741.0
1,Future People,201612,False,Future People,0,Alabama Shakes,"['alabama indie', 'modern blues rock', 'rock']",1604741.0
2,Sound & Color,232546,True,Don't Wanna Fight,0,Alabama Shakes,"['alabama indie', 'modern blues rock', 'rock']",1604741.0
3,Sound & Color,243466,False,Gimme All Your Love,0,Alabama Shakes,"['alabama indie', 'modern blues rock', 'rock']",1604741.0
4,Arrow,308066,False,Only For You,3,Heartless Bastards,['stomp and holler'],179423.0
...,...,...,...,...,...,...,...,...
95,Una Volta,330419,False,Queen of the Surface Streets,0,DeVotchKa,NaN,NaN
96,Lonerism,211226,False,Elephant,75,Tame Impala,"['australian psych', 'modern rock', 'neo-psych...",7453237.0
97,Had Ten Dollaz,252139,False,Had Ten Dollaz,46,Cherry Glazerr,"['bubblegrunge', 'indie punk', 'indie surf', '...",164302.0
98,100 Lovers,290106,False,All the Sand in all the Sea,35,DeVotchKa,NaN,NaN


In [593]:
songsAnalysis = songs.drop(['uri', 'id', 'href', 'external_ids', 'available_markets', 'track_number'], axis=1)

In [594]:
songsAlbums = songsAnalysis['album']

In [595]:
songsAlbumsArr = []
for i in songsAlbums:
    songsAlbumsArr.append(ast.literal_eval(i)['name'])

In [596]:
songsAnalysis['album'] = songsAlbumsArr

In [597]:
songsArtists = songsAnalysis['artists']

In [598]:
songsArtistsArr = []
mainArtistsArr = []
for i in songsArtists:
    artistsArr = []
    for artist in ast.literal_eval(i):
        artistsArr.append(artist['name'])
    songsArtistsArr.append(artistsArr)
    mainArtistsArr.append(ast.literal_eval(i)[0]['name'])

In [599]:
songsAnalysis['artists'] = songsArtistsArr
songsAnalysis['main_artist'] = mainArtistsArr

In [600]:
songsAnalysis['album'].value_counts()

Greatest Hits                                                            85
Original Album Classics                                                  54
Les Misérables: The Motion Picture Soundtrack Deluxe (Deluxe Edition)    42
World Classical 2010                                                     42
Frozen (Original Motion Picture Soundtrack / Deluxe Edition)             36
                                                                         ..
The Music of DC Comics: Volume 2                                          1
Mission Impossible (Original Motion Picture Soundtrack)                   1
Fast & Furious! (Epic Action Movie Soundtrack Songs)                      1
Dr. No (Original Soundtrack) [1962]                                       1
Cosmic Angel (Acoustic From Capitol Studios)                              1
Name: album, Length: 17951, dtype: int64

In [601]:
songsAnalysis['artists'].value_counts()

[Pearl Jam]                                                                                                                                                159
[Tegan and Sara]                                                                                                                                           138
[John Mayer]                                                                                                                                                95
[Deftones]                                                                                                                                                  78
[Drake]                                                                                                                                                     78
                                                                                                                                                          ... 
[Gavin Creel, Marc Kudisch, Sutton Foster, Ang

In [602]:
songsAnalysis['album'].nunique()

17951

In [603]:
artists['main_artist'] = artists['name']

In [604]:
songsAnalysis = pd.merge(songsAnalysis, artists[['main_artist', 'genres', 'followers']], on='main_artist', how='left')

In [605]:
songsAnalysis = songsAnalysis.sort_values(by=['popularity'], ascending=False)
songsAnalysis.drop_duplicates(subset=['name'], keep='first', inplace=True)
songsAnalysis = songsAnalysis.drop('artists', axis=1)

In [606]:
scaler = MinMaxScaler()
numerical_columns = songsAnalysis.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_columns = songsAnalysis.drop('explicit', axis=1).select_dtypes(include=['object']).columns.tolist()
binary_columns = songsAnalysis['explicit']
songsAnalysis[numerical_columns] = scaler.fit_transform(songsAnalysis[numerical_columns])

In [607]:
songsAnalysis = songsAnalysis.join(pd.get_dummies(songsAnalysis['explicit']))
songsAnalysis.dropna(inplace=True)
songsAnalysis.reset_index(inplace=True)
songsAnalysis.drop(['explicit', 'index'], axis=1, inplace=True)

In [608]:
songsAnalysis

,album,duration_ms,name,popularity,main_artist,genres,followers,False,True
0,Parachutes,0.110046,Yellow,1.00000,Coldplay,"['permanent wave', 'pop']",0.434773,1,0
1,I Love You.,0.099167,Sweater Weather,1.00000,The Neighbourhood,"['modern alternative rock', 'modern rock', 'pop']",0.137250,1,0
2,Long Way Down (Deluxe),0.100801,Another Love,0.98913,Tom Odell,['chill pop'],0.032803,0,1
3,Blonde,0.076115,Pink + White,0.98913,Frank Ocean,"['lgbtq+ hip hop', 'neo soul']",0.116047,1,0
4,Flower Boy,0.074411,See You Again (feat. Kali Uchis),0.98913,"Tyler, The Creator","['hip hop', 'rap']",0.126035,0,1
...,...,...,...,...,...,...,...,...,...
30054,In Return,0.144003,For Us,0.00000,ODESZA,"['chillwave', 'edm', 'indietronica']",0.013969,1,0
30055,In Return,0.102302,Memories That You Call,0.00000,ODESZA,"['chillwave', 'edm', 'indietronica']",0.013969,1,0
30056,Drunk Enough to Say That I Love You,0.091685,Drunk Enough to Say That I Love You,0.00000,PLVTINUM,['dark r&b'],0.002837,1,0
30057,Motown Christmas,0.105619,Go Tell It On The Mountain / Ain't No Mountain...,0.00000,Gregory Porter,"['contemporary vocal jazz', 'jazz pop', 'vocal...",0.005553,1,0


In [609]:
genreList = []
for genre in songsAnalysis['genres']:
    genreList.append(genre)

In [610]:
uniqueGenres = []
for i in range(len(genreList)):
    for genre in ast.literal_eval(genreList[i]):
        uniqueGenres.append(genre)

In [611]:
uniqueGenres = pd.DataFrame(uniqueGenres)
genre_counts = uniqueGenres[0].value_counts()
uniqueGenres = pd.DataFrame(genre_counts)
uniqueGenres.reset_index(inplace=True)
uniqueGenres.columns = ['genre', 'count']
uniqueGenres.sort_values(by='count', ascending=False, inplace=True)
uniqueGenres.drop_duplicates(keep='first', inplace=True)
uniqueGenres.reset_index(inplace=True)
uniqueGenres.drop('index', axis=1, inplace=True)

In [612]:
def find_most_common_genre(genre_list):
    priority_threshold = 20  # Set your priority threshold here

    if len(genre_list) == 0:
        return None

    # Iterate through each genre in the list
    most_common_genre = None
    lowest_index = float('inf')  # Initialize with positive infinity
    for genre in ast.literal_eval(genre_list):
        if genre in uniqueGenres['genre'].values:
            index_in_sorted = uniqueGenres.index[uniqueGenres['genre'] == genre].tolist()[0]
            if index_in_sorted < lowest_index:
                lowest_index = index_in_sorted
                most_common_genre = genre
            elif index_in_sorted - lowest_index <= priority_threshold:
                most_common_genre = genre
    return most_common_genre

# Apply the function to each row in your dataset
songsAnalysis['single_genre'] = songsAnalysis['genres'].apply(find_most_common_genre)

In [ ]:
songsAnalysis

,album,duration_ms,name,popularity,main_artist,genres,followers,False,True,single_genre
0,Parachutes,0.110046,Yellow,1.00000,Coldplay,"['permanent wave', 'pop']",0.434773,1,0,pop
1,I Love You.,0.099167,Sweater Weather,1.00000,The Neighbourhood,"['modern alternative rock', 'modern rock', 'pop']",0.137250,1,0,pop
2,Long Way Down (Deluxe),0.100801,Another Love,0.98913,Tom Odell,['chill pop'],0.032803,0,1,None
3,Blonde,0.076115,Pink + White,0.98913,Frank Ocean,"['lgbtq+ hip hop', 'neo soul']",0.116047,1,0,neo soul
4,Flower Boy,0.074411,See You Again (feat. Kali Uchis),0.98913,"Tyler, The Creator","['hip hop', 'rap']",0.126035,0,1,rap
...,...,...,...,...,...,...,...,...,...,...
30054,In Return,0.144003,For Us,0.00000,ODESZA,"['chillwave', 'edm', 'indietronica']",0.013969,1,0,indietronica
30055,In Return,0.102302,Memories That You Call,0.00000,ODESZA,"['chillwave', 'edm', 'indietronica']",0.013969,1,0,indietronica
30056,Drunk Enough to Say That I Love You,0.091685,Drunk Enough to Say That I Love You,0.00000,PLVTINUM,['dark r&b'],0.002837,1,0,None
30057,Motown Christmas,0.105619,Go Tell It On The Mountain / Ain't No Mountain...,0.00000,Gregory Porter,"['contemporary vocal jazz', 'jazz pop', 'vocal...",0.005553,1,0,vocal jazz


In [ ]:
tsne = TSNE(n_components=2)
tsne_data = tsne.fit_transform(songsAnalysis)

ValueError: could not convert string to float: 'Parachutes'

In [ ]:
hierarchical_cluster = AgglomerativeClustering(n_clusters=None, distance_threshold=0.5, linkage='ward')
clusters = hierarchical_cluster.fit_predict(songsAnalysis)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').